In [105]:
import pandas as pd
from scipy import stats
import random
from datetime import datetime as dt, timedelta
import numpy as np

import sys 
import os 
sys.path.insert(0, os.path.dirname(os.getcwd()))

random.seed(42)

In [106]:
df_history = pd.read_csv('../data/sales_history.csv')
df_forecast = pd.read_csv('../data/sales_forecast.csv')
df_hub_stock = pd.read_csv('../data/hub_stock.csv')


In [141]:
# parameters: 
config = {
    "lead_time_hud2channel": 5,
    "lead_time_plant2hub": 10,
    "safety_stock_factor" : {
        "A" : 10,
        "B" : 10, 
        "C": 10
    },
    "channel_plan":{
        "channel 1" :{
            "priority" : 1,
            "service_level" : 0.8
        },        
        "channel 2" :{
            "priority" : 2,
            "service_level" : 0.8
        },
        "channel 3" :{
            "priority" : 3,
            "service_level" : 0.8
        },
        "channel 4" :{
            "priority" : 4,
            "service_level" : 0.8
            
            
        },
    }
}



sku_ls = ['A', 'B', 'C']
channel_ls = ['channel 1', 'channel 2', 'channel 3', 'channel 4']

current_date = '2023-01-14'
start_date = '2023-01-15'


import simulation
from importlib import reload 
reload(simulation)
import xlwings as xw

from simulation import generate_simulation,get_status,ttl_sales_through_rate,ttl_sales_shortage_rate

df_allocation = generate_simulation(
    config, 
    sku_ls,'2023-01-14', '2023-01-15', df_history, df_forecast, df_hub_stock, [1,2,3,4]
)
# df_allocation.to_excel('../data/allocation.xlsx', index= False)

df_allocation['status'] = df_allocation.apply(get_status, axis = 1)
df_allocation['date'] = pd.to_datetime(df_allocation['date'])


df_status = pd.pivot_table(df_allocation, index=['sku_name', 'channel'],columns= 'date' ,values='status', aggfunc=lambda x: ', '.join(x))
sales_through_breakdown = df_allocation.groupby(['sku_name','channel']).apply(lambda x: x.sales.sum()/(x['stock_in'].sum() + x[x['week']==0].stock) )
stockout_breakdown = df_allocation.groupby(['sku_name','channel']).apply(lambda x: pd.Series(1 - x.sales.sum()/x.sales_potential_daily.sum(), index = ['stockout']))
sales_through_breakdown.columns = ['sales_through']
stockout_breakdown.columns = ['stockout']
df_status = sales_through_breakdown.join(stockout_breakdown).join(df_status)

# Connects to the active instance of Excel
wb = xw.Book('../data/allocation.xlsx')  # Connects to the active instance of Excel
sheet = wb.sheets['Sheet1']
sheet.range("A:L")[1:,:].clear_contents()
sheet['A1'].options(index=False).value = df_allocation
sheet = wb.sheets['Sheet2']
sheet.range('A3').value = df_status


sheet.range('A1').value = 'sale through rate:'
sheet.range('C1').value = 'stockout rate:'
sheet.range('A2').value = ttl_sales_through_rate(df_allocation)
sheet.range('C2').value = ttl_sales_shortage_rate(df_allocation)


/var/folders/nx/d8ckq9qd7l3dz5plmv5gdj_40000gn/T/ipykernel_14531/3199533630.py:59: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

/var/folders/nx/d8ckq9qd7l3dz5plmv5gdj_40000gn/T/ipykernel_14531/3199533630.py:60: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [42]:
# sales_demand * service_level     +    safety_stock 

# sales_demand * service_level     +    sales_demand/28 * factor

# sales_demand * (service_level + factor/28)

## Search

In [43]:
from itertools import permutations,product
from simulation import generate_simulation,get_status,ttl_sales_through_rate,ttl_sales_shortage_rate

In [285]:
search_space = {
    'lead_time_hud2channel' : [3,5],
    'lead_time_plant2hub' : [10, 15],
    'priority' : [1,2,3,4], 
    'service_level' : [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
}


priority_permu = list(permutations(search_space['priority'], 4))


combinations = list(
    product(   
        search_space['lead_time_hud2channel'], 
        search_space['lead_time_plant2hub'], 
        search_space['service_level'], 
        priority_permu
    )
)

sku_ls = ['A', 'B', 'C']

config_spaces = [
    {
        "trial": i,
        "lead_time_hud2channel": c[0],
        "lead_time_plant2hub": c[1],
        "safety_stock_factor" : {
            "A" : 10,
            "B" : 10, 
            "C": 15
        },
        "channel_plan":{
            "channel 1" :{
                "priority" : c[3][0],
                "service_level" : c[2]
            },        
            "channel 2" :{
                "priority" : c[3][1],
                "service_level" : c[2]
            },
            "channel 3" :{
                "priority" : c[3][2],
                "service_level" : c[2]
            },
            "channel 4" :{
                "priority" : c[3][3],
                "service_level" : c[2]
            },
        }
    }
    for i,c in enumerate(combinations)
]

In [294]:
from multiprocess import Pool
import multiprocess as mp
n_process = mp.cpu_count()
print(n_process)


8


In [292]:
def simulate_one_trails(config_,df_history, df_forecast, df_hub_stock):
    df_allocation_ = generate_simulation(
        config_, 
        sku_ls,'2023-01-14', '2023-01-15', df_history, df_forecast, df_hub_stock, [1,2,3,4]
    )
    sales_though  = ttl_sales_through_rate(df_allocation_)
    fill_rate =  1- ttl_sales_shortage_rate(df_allocation_)
    
    channel_plan = config_['channel_plan']
    channel_order = sorted(channel_plan, key=lambda x: channel_plan[x]['priority'])

    order = ''
    for o in channel_order:
        order += f"{o}({int(config_['channel_plan'][o]['service_level'] * 100)}), "

    df_config_ = pd.DataFrame(
        {
            "trial": [config_['trial']],
            "sales through": [sales_though],
            "fill rate": [fill_rate],
            "transportation days (to hub)" : [config_['lead_time_plant2hub']],
            "transportation days (to channel)" : [config_['lead_time_hud2channel']],
            "safety_stock_factor": [config_['safety_stock_factor']],
            "service level and order": [order]
        }
    )
    
    return df_config_

In [293]:
simulate_one_trails(config_spaces[0])

,trial,sales through,fill rate,transportation days (to hub),transportation days (to channel),safety_stock_factor,service level and order
0,0,0.994721,0.959154,10,3,"{'A': 10, 'B': 10, 'C': 15}","channel 1(60), channel 2(60), channel 3(60), c..."


In [ ]:

from tqdm import tqdm 


def search_multi_process(kwargls, func, n_process = None):
    if not n_process:
        n_process = mp.cpu_count()-1
        
    pool = Pool(n_process)
    l = list()
    for i in tqdm(pool.imap_unordered(func, kwargls), total = len(kwargls)):
        l.append(i)
    return l




In [284]:
from tqdm import tqdm 
n_process = mp.cpu_count()
print(n_process)

sales_through_ls = []
fill_rate_ls = []

df_result = pd.DataFrame()
for i, config_ in tqdm(enumerate(config_spaces)):

    df_allocation_ = generate_simulation(
        config_, 
        sku_ls,'2023-01-14', '2023-01-15', df_history, df_forecast, df_hub_stock, [1,2,3,4]
    )
    sales_though  = ttl_sales_through_rate(df_allocation_)
    fill_rate =  1- ttl_sales_shortage_rate(df_allocation_)
    
    channel_plan = config_['channel_plan']
    channel_order = sorted(channel_plan, key=lambda x: channel_plan[x]['priority'])

    order = ''
    for o in channel_order:
        order += f"{o}({int(config_['channel_plan'][o]['service_level'] * 100)}), "

    df_config_ = pd.DataFrame(
        {
            "trials": [i],
            "sales through": [sales_though],
            "fill rate": [fill_rate],
            "transportation days (to hub)" : [config_['lead_time_plant2hub']],
            "transportation days (to channel)" : [config_['lead_time_hud2channel']],
            "safety_stock_factor": [config_['safety_stock_factor']],
            "service level and order": [order]
        }
    )
    
    
    df_result = pd.concat([df_result, df_config_], axis = 0)


8


768it [00:42, 18.20it/s]


In [282]:
channel_order

['channel 1', 'channel 2', 'channel 3', 'channel 4']

,trials,sales through,fill rate,transportation days (to hub),transportation days (to channel),safety_stock_factor,service level and order
0,0,0.994721,0.959154,10,3,"{'A': 10, 'B': 10, 'C': 15}","channel 1(60), channel 2(60), channel 3(60), c..."


In [104]:
import plotly.express as px
fig = px.scatter(x=sales_through_ls, y=fill_rate_ls,hover_name = [i for i in range(len(config_spaces))],opacity= .8 )

fig.update_xaxes(mirror = True, title = 'sale through')
fig.update_yaxes(mirror = True, title = 'fill rate')
fig.update_layout(
    template = 'simple_white', height = 600, width = 700,
)



In [100]:
config_spaces[743]

{'lead_time_hud2channel': 5,
 'lead_time_plant2hub': 15,
 'safety_stock_factor': {'A': 3, 'B': 10, 'C': 15},
 'channel_plan': {'channel 1': {'priority': 4, 'service_level': 0.9},
  'channel 2': {'priority': 3, 'service_level': 0.9},
  'channel 3': {'priority': 2, 'service_level': 0.9},
  'channel 4': {'priority': 1, 'service_level': 0.9}}}

In [82]:
28 * 0.6

16.8

In [83]:
12 

12

In [101]:
import xlwings as xw

df_allocation = generate_simulation(
    config_spaces[743], 
    sku_ls,'2023-01-14', '2023-01-15', df_history, df_forecast, df_hub_stock, [1,2,3,4]
)
# df_allocation.to_excel('../data/allocation.xlsx', index= False)

df_allocation['status'] = df_allocation.apply(get_status, axis = 1)
df_allocation['date'] = pd.to_datetime(df_allocation['date'])


df_status = pd.pivot_table(df_allocation, index=['sku_name', 'channel'],columns= 'date' ,values='status', aggfunc=lambda x: ', '.join(x))
sales_through_breakdown = df_allocation.groupby(['sku_name','channel']).apply(lambda x: x.sales.sum()/(x['stock_in'].sum() + x[x['week']==0].stock) )
stockout_breakdown = df_allocation.groupby(['sku_name','channel']).apply(lambda x: pd.Series(1 - x.sales.sum()/x.sales_potential_daily.sum(), index = ['stockout']))
sales_through_breakdown.columns = ['sales_through']
stockout_breakdown.columns = ['stockout']
df_status = sales_through_breakdown.join(stockout_breakdown).join(df_status)

# Connects to the active instance of Excel
wb = xw.Book('../data/allocation.xlsx')  # Connects to the active instance of Excel
sheet = wb.sheets['Sheet1']
sheet.range("A:L")[1:,:].clear_contents()
sheet['A1'].options(index=False).value = df_allocation
sheet = wb.sheets['Sheet2']
sheet.range('A3').value = df_status


sheet.range('A1').value = 'sale through rate:'
sheet.range('C1').value = 'stockout rate:'
sheet.range('A2').value = ttl_sales_through_rate(df_allocation)
sheet.range('C2').value = ttl_sales_shortage_rate(df_allocation)

/var/folders/nx/d8ckq9qd7l3dz5plmv5gdj_40000gn/T/ipykernel_14531/1389068161.py:14: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

/var/folders/nx/d8ckq9qd7l3dz5plmv5gdj_40000gn/T/ipykernel_14531/1389068161.py:15: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



['sku_name', 'channel', 'sales_through', 'stockout']

In [134]:
from datetime import datetime as dt 
import dash_ag_grid as dag

def _status2table(df_status):
    df_status_ = df_status.reset_index()
    datetime_cols = [i.strftime("%Y %b %d") for i in df_status_.columns[4:]]
    df_status_.columns = list(df_status_.columns[:4]) + datetime_cols

    columnDef = [
        {'field': c}
        for c in df_status_.columns
    ]

    status_table = dag.AgGrid(
            id='status-table',
            rowData=df_status_.to_dict('records'),
            columnDefs=columnDef,
            defaultColDef={"filter": False},
            columnSize="sizeToFit",
            style= {
                'height':240, 'font-size':'12rem'
            }
    )
    
    return status_table

,sku_name,channel,sales_through,stockout,2023 Jan 14,2023 Jan 15,2023 Jan 16,2023 Jan 17,2023 Jan 18,2023 Jan 19,...,2023 Feb 02,2023 Feb 03,2023 Feb 04,2023 Feb 05,2023 Feb 06,2023 Feb 07,2023 Feb 08,2023 Feb 09,2023 Feb 10,2023 Feb 11
0,A,channel 1,0.863878,0.000000,risk,risk,risk,near stockout,near stockout,near stockout,...,full,full,full,full,risk,risk,risk,risk,risk,near stockout
1,A,channel 2,0.864080,0.000000,risk,risk,risk,near stockout,near stockout,near stockout,...,full,full,full,full,risk,risk,risk,risk,risk,near stockout
2,A,channel 3,0.863960,0.000000,risk,risk,risk,near stockout,near stockout,near stockout,...,full,full,full,full,risk,risk,risk,risk,risk,near stockout
3,A,channel 4,0.810586,0.061427,risk,risk,risk,risk,near stockout,near stockout,...,full,full,full,full,full,risk,risk,risk,risk,risk
4,B,channel 1,0.863179,0.000000,risk,risk,risk,near stockout,near stockout,near stockout,...,full,full,full,full,risk,risk,risk,risk,risk,near stockout
5,B,channel 2,0.864101,0.000000,risk,risk,risk,near stockout,near stockout,near stockout,...,full,full,full,full,risk,risk,risk,risk,risk,near stockout
6,B,channel 3,0.863176,0.000000,risk,risk,risk,near stockout,near stockout,near stockout,...,full,full,full,risk,risk,risk,risk,risk,risk,near stockout
7,B,channel 4,0.862372,0.000000,risk,risk,risk,near stockout,near stockout,near stockout,...,full,full,full,full,risk,risk,risk,risk,risk,near stockout
8,C,channel 1,0.853933,0.000000,risk,risk,risk,near stockout,near stockout,near stockout,...,full,full,full,risk,risk,risk,risk,risk,risk,near stockout
9,C,channel 2,0.863501,0.000000,risk,risk,risk,near stockout,near stockout,near stockout,...,full,full,full,full,risk,risk,risk,risk,risk,near stockout


[{'sku_name': 'A',
  'channel': 'channel 1',
  'sales_through': 0.8638781431334621,
  'stockout': 0.0,
  '2023 Jan 14': 'risk',
  '2023 Jan 15': 'risk',
  '2023 Jan 16': 'risk',
  '2023 Jan 17': 'near stockout',
  '2023 Jan 18': 'near stockout',
  '2023 Jan 19': 'near stockout',
  '2023 Jan 20': 'hub',
  '2023 Jan 21': 'full',
  '2023 Jan 22': 'full',
  '2023 Jan 23': 'full',
  '2023 Jan 24': 'full',
  '2023 Jan 25': 'full',
  '2023 Jan 26': 'full',
  '2023 Jan 27': 'full',
  '2023 Jan 28': 'full',
  '2023 Jan 29': 'full',
  '2023 Jan 30': 'full',
  '2023 Jan 31': 'full',
  '2023 Feb 01': 'full',
  '2023 Feb 02': 'full',
  '2023 Feb 03': 'full',
  '2023 Feb 04': 'full',
  '2023 Feb 05': 'full',
  '2023 Feb 06': 'risk',
  '2023 Feb 07': 'risk',
  '2023 Feb 08': 'risk',
  '2023 Feb 09': 'risk',
  '2023 Feb 10': 'risk',
  '2023 Feb 11': 'near stockout'},
 {'sku_name': 'A',
  'channel': 'channel 2',
  'sales_through': 0.8640796019900497,
  'stockout': 0.0,
  '2023 Jan 14': 'risk',
  '2023 

In [126]:
datetime

['2023 00 14',
 '2023 00 15',
 '2023 00 16',
 '2023 00 17',
 '2023 00 18',
 '2023 00 19',
 '2023 00 20',
 '2023 00 21',
 '2023 00 22',
 '2023 00 23',
 '2023 00 24',
 '2023 00 25',
 '2023 00 26',
 '2023 00 27',
 '2023 00 28',
 '2023 00 29',
 '2023 00 30',
 '2023 00 31',
 '2023 00 01',
 '2023 00 02',
 '2023 00 03',
 '2023 00 04',
 '2023 00 05',
 '2023 00 06',
 '2023 00 07',
 '2023 00 08',
 '2023 00 09',
 '2023 00 10',
 '2023 00 11']

[Timestamp('2023-01-14 00:00:00'),
 Timestamp('2023-01-15 00:00:00'),
 Timestamp('2023-01-16 00:00:00'),
 Timestamp('2023-01-17 00:00:00'),
 Timestamp('2023-01-18 00:00:00'),
 Timestamp('2023-01-19 00:00:00'),
 Timestamp('2023-01-20 00:00:00'),
 Timestamp('2023-01-21 00:00:00'),
 Timestamp('2023-01-22 00:00:00'),
 Timestamp('2023-01-23 00:00:00'),
 Timestamp('2023-01-24 00:00:00'),
 Timestamp('2023-01-25 00:00:00'),
 Timestamp('2023-01-26 00:00:00'),
 Timestamp('2023-01-27 00:00:00'),
 Timestamp('2023-01-28 00:00:00'),
 Timestamp('2023-01-29 00:00:00'),
 Timestamp('2023-01-30 00:00:00'),
 Timestamp('2023-01-31 00:00:00'),
 Timestamp('2023-02-01 00:00:00'),
 Timestamp('2023-02-02 00:00:00'),
 Timestamp('2023-02-03 00:00:00'),
 Timestamp('2023-02-04 00:00:00'),
 Timestamp('2023-02-05 00:00:00'),
 Timestamp('2023-02-06 00:00:00'),
 Timestamp('2023-02-07 00:00:00'),
 Timestamp('2023-02-08 00:00:00'),
 Timestamp('2023-02-09 00:00:00'),
 Timestamp('2023-02-10 00:00:00'),
 Timestamp('2023-02-

In [110]:
import openpyxl, io

# Create an in-memory bytes buffer, which is a file-like object
buffer = io.BytesIO()

# Create a workbook, do the shit you gotta do
wb = openpyxl.Workbook('../data/allocation.xlsx')

# Save the workbook to the buffer, as if it were a file on disk opened for writing bytes
wb.save(buffer)

# Get all the bytes in the buffer.
# It's functionally the same as buffer.seek(0) then buffer.read()
a = buffer.getvalue()

In [113]:
with open('tmp.xlsx', 'wb') as f:
    f.write(a)

In [117]:
import re 

re.findall(r'\d+', 'Sea(20 days)')[0]

'20'

In [162]:
df_allocation_selected['stock'].diff().to_list()

[nan,
 -74.85714285714289,
 -74.85714285714289,
 -74.85714285714289,
 -74.85714285714289,
 -74.85714285714286,
 -74.85714285714286,
 -74.85714285714286,
 -66.57142857142857,
 -9.428571428571288,
 0.0,
 1665.4285714285713,
 -66.57142857142867,
 -66.57142857142867,
 -66.57142857142867,
 -77.28571428571422,
 -77.28571428571422,
 -77.28571428571422,
 -77.28571428571422,
 -77.28571428571422,
 -77.28571428571433,
 -77.28571428571433,
 -69.0,
 -69.0,
 -69.0,
 -69.0,
 -69.0,
 -69.0,
 -69.0]

In [170]:
df_allocation_selected.columns

Index(['sku_name', 'channel', 'date', 'week', 'stock_in', 'stock_from',
       'stock', 'sales_potential_weekly', 'sales_potential_daily', 'sales',
       'safety_stock', 'alert', 'status'],
      dtype='object')

In [255]:
x = '2023-01-20'

In [259]:
import plotly.graph_objects as go 



fig = go.Figure()

fig.add_hline(y = df_allocation_selected['safety_stock'].values[0], line = dict(color = '#E99497', width = 1, dash = 'dot'))
fig.add_trace(
    go.Waterfall(
    name = "20", orientation = "v",
    x = df_allocation_selected['date'],
    textposition = "outside",
    # text = df_allocation_selected['status'],
    # decreasing = {"marker":{"color":"#F7D060"}},
    increasing = {"marker":{"color":"#F7D060"}},
    y = [df_allocation_selected['stock'].values[0]] + df_allocation_selected['stock'].diff().tolist()[1:],
    connector = {"line":{"color":"rgb(63, 63, 63)"}},
))

fig.add_annotation(
    x = 1, y = df_allocation_selected['safety_stock'].values[0],
    xref='paper', yref='y',
    xanchor='left',
    align='left',
    text= 'Safety<br>Stock<br>Level',
    showarrow=False,
    font = {'color': '#E99497'}
)

max_stock = df_allocation_selected['stock'].max()
for i, d in df_allocation_selected.reset_index().iterrows():
    if d['status'] not in ['risk', 'full', 'near stockout'] or i == 0:
        fig.add_annotation(
            x = d['date'].strftime("%Y-%m-%d"), y = d['stock'] + max_stock*0.05,
            # xanchor='',
            align='center',
            text= 'current<br>stock'if i==0 else d['status'],
            showarrow=True,arrowhead = 2
        )

from datetime import timedelta


fig.add_shape(type="rect",
    x0=x, y0=0, x1=x, y1=max_stock * 1.2,
    line=dict(
        color="RoyalBlue",
        width=2,
    ),
    fillcolor="LightSkyBlue",
)


fig.update_layout(template = 'simple_white',showlegend = False, height = 400, margin = {'t':0, 'l':0,'b':0, 'r':0})
fig.update_xaxes(mirror = True,)
fig.update_yaxes(mirror = True, title = 'stock', range = [0, max_stock * 1.2])

fig.show()


1674518400000000000

In [168]:
fig.data

(Waterfall({
     'connector': {'line': {'color': 'rgb(63, 63, 63)'}},
     'name': '20',
     'orientation': 'v',
     'textposition': 'outside',
     'x': array([datetime.datetime(2023, 1, 14, 0, 0),
                 datetime.datetime(2023, 1, 15, 0, 0),
                 datetime.datetime(2023, 1, 16, 0, 0),
                 datetime.datetime(2023, 1, 17, 0, 0),
                 datetime.datetime(2023, 1, 18, 0, 0),
                 datetime.datetime(2023, 1, 19, 0, 0),
                 datetime.datetime(2023, 1, 20, 0, 0),
                 datetime.datetime(2023, 1, 21, 0, 0),
                 datetime.datetime(2023, 1, 22, 0, 0),
                 datetime.datetime(2023, 1, 23, 0, 0),
                 datetime.datetime(2023, 1, 24, 0, 0),
                 datetime.datetime(2023, 1, 25, 0, 0),
                 datetime.datetime(2023, 1, 26, 0, 0),
                 datetime.datetime(2023, 1, 27, 0, 0),
                 datetime.datetime(2023, 1, 28, 0, 0),
                 datetime.da

In [235]:
df_allocation_selected = df_allocation.query(f"sku_name == 'A' & channel == 'channel 3'")
df_allocation_selected

,sku_name,channel,date,week,stock_in,stock_from,stock,sales_potential_weekly,sales_potential_daily,sales,safety_stock,alert,status
0,A,channel 3,2023-01-14,0.0,0.0,,800.000000,NaN,NaN,NaN,1042.0,1,risk
0,A,channel 3,2023-01-15,1.0,0.0,,698.285714,712.0,101.714286,101.714286,1042.0,1,risk
1,A,channel 3,2023-01-16,1.0,0.0,,596.571429,712.0,101.714286,101.714286,1042.0,1,risk
2,A,channel 3,2023-01-17,1.0,0.0,,494.857143,712.0,101.714286,101.714286,1042.0,1,near stockout
3,A,channel 3,2023-01-18,1.0,0.0,,393.142857,712.0,101.714286,101.714286,1042.0,1,near stockout
4,A,channel 3,2023-01-19,1.0,0.0,,291.428571,712.0,101.714286,101.714286,1042.0,1,near stockout
5,A,channel 3,2023-01-20,1.0,1039.0,hub,1228.714286,712.0,101.714286,101.714286,1042.0,0,hub
6,A,channel 3,2023-01-21,1.0,0.0,,1127.000000,712.0,101.714286,101.714286,1042.0,0,full
7,A,channel 3,2023-01-22,2.0,0.0,,1020.000000,749.0,107.000000,107.000000,1042.0,1,risk
8,A,channel 3,2023-01-23,2.0,0.0,,913.000000,749.0,107.000000,107.000000,1042.0,1,risk
